In [ ]:
ls
mkdir datasets
mkdir models
ls

In [10]:
# 0 the destination of the zip files
complete_dataset_url = 'http://files.grouplens.org/datasets/movielens/ml-latest.zip'
small_dataset_url = 'http://files.grouplens.org/datasets/movielens/ml-latest-small.zip'


# 0 define the download path of the files
import os

datasets_path = os.path.join('.', 'datasets')

complete_dataset_path = os.path.join(datasets_path, 'ml-latest.zip')
small_dataset_path = os.path.join(datasets_path, 'ml-latest-small.zip')


# 0 Download the files
from urllib.request import urlretrieve

small_f =  urlretrieve(small_dataset_url, small_dataset_path)
complete_f =  urlretrieve(complete_dataset_url, complete_dataset_path)


# 0 tar the files.
import zipfile

with zipfile.ZipFile(small_dataset_path, "r") as z:
    z.extractall(datasets_path)

with zipfile.ZipFile(complete_dataset_path, "r") as z:
    z.extractall(datasets_path)

In [22]:
ls ./datasets

ml-latest/  ml-latest-small/  ml-latest-small.zip  ml-latest.zip


The start of data 

function used: take

In [23]:
# 1 rating: creat RDD
small_ratings_file = os.path.join(datasets_path, 'ml-latest-small', 'ratings.csv')

small_ratings_raw_data = sc.textFile(small_ratings_file)
small_ratings_raw_data_header = small_ratings_raw_data.take(1)[0]

In [26]:
small_ratings_raw_data.take(5)

['userId,movieId,rating,timestamp',
 '1,31,2.5,1260759144',
 '1,1029,3.0,1260759179',
 '1,1061,3.0,1260759182',
 '1,1129,2.0,1260759185']

In [27]:
# 1 filter the lines, split the words and take only the first 3 columns
small_ratings_data = small_ratings_raw_data.filter(lambda line: line!=small_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1],tokens[2])).cache()

In [28]:
small_ratings_data.take(3)

[('1', '31', '2.5'), ('1', '1029', '3.0'), ('1', '1061', '3.0')]

In [29]:
# 1 movie: create, filter RDD
small_movies_file = os.path.join(datasets_path, 'ml-latest-small', 'movies.csv')

small_movies_raw_data = sc.textFile(small_movies_file)
small_movies_raw_data_header = small_movies_raw_data.take(1)[0]

small_movies_data = small_movies_raw_data.filter(lambda line: line!=small_movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (tokens[0],tokens[1])).cache()
    
small_movies_data.take(3)

[('1', 'Toy Story (1995)'),
 ('2', 'Jumanji (1995)'),
 ('3', 'Grumpier Old Men (1995)')]

In [30]:
# 2  sample subset, and map
training_RDD, validation_RDD, test_RDD = small_ratings_data.randomSplit([6, 2, 2], seed=123)
validation_for_predict_RDD = validation_RDD.map(lambda x: (x[0], x[1]))
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

In [32]:
validation_RDD.take(2)

[('1', '2193', '2.0'), ('2', '17', '5.0')]

In [31]:
validation_for_predict_RDD.take(2)

[('1', '2193'), ('2', '17')]

used RDD.join(RDD)
used RDD.mean()

In [33]:
# 2 Fit the model, and validation to tune the hyperparameters
from pyspark.mllib.recommendation import ALS
import math

seed = 532
iterations = 10
regularization_parameter = 0.1
ranks = [4, 8, 12]
errors = [0, 0, 0]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1
for rank in ranks:
    model = ALS.train(training_RDD, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validation_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    errors[err] = error
    err += 1
    print('For rank %s the RMSE is %s' % (rank, error))
    if error < min_error:
        min_error = error
        best_rank = rank

print('The best model was trained with rank %s' % best_rank)

For rank 4 the RMSE is 0.9400020429974122
For rank 8 the RMSE is 0.9468288439560869
For rank 12 the RMSE is 0.9500950485748263
The best model was trained with rank 4


In [37]:
predictions.take(3)

[((452, 1084), 3.819532706475835),
 ((30, 1084), 4.106375714282098),
 ((278, 1084), 3.4028377645187464)]

In [38]:
rates_and_preds.take(3)

[((472, 2014), (3.0, 3.864299195240666)),
 ((254, 48), (1.0, 2.368551117192621)),
 ((558, 3728), (5.0, 3.8831056163918083))]

In [39]:
# 2 Retrain the model based on best hyperparameter, and test data.
model = ALS.train(training_RDD, best_rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
predictions = model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print('For testing data the RMSE is %s' % (error))

For testing data the RMSE is 0.937826791033363


used RDD.count()

In [40]:
# 3 Load the complete dataset file
complete_ratings_file = os.path.join(datasets_path, 'ml-latest', 'ratings.csv')
complete_ratings_raw_data = sc.textFile(complete_ratings_file)
complete_ratings_raw_data_header = complete_ratings_raw_data.take(1)[0]

# Parse
complete_ratings_data = complete_ratings_raw_data.filter(lambda line: line!=complete_ratings_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),int(tokens[1]),float(tokens[2]))).cache()
    
print ("There are %s recommendations in the complete dataset" % (complete_ratings_data.count()))

There are 26024289 recommendations in the complete dataset


In [41]:
# 3 Train the complete dataset. 
training_RDD, test_RDD = complete_ratings_data.randomSplit([7, 3], seed=12)

complete_model = ALS.train(training_RDD, best_rank, seed=seed, 
                           iterations=iterations, lambda_=regularization_parameter)

In [53]:
# 3 predict the test set, cal error
test_for_predict_RDD = test_RDD.map(lambda x: (x[0], x[1]))

predictions = complete_model.predictAll(test_for_predict_RDD).map(lambda r: ((r[0], r[1]), r[2]))
rates_and_preds = test_RDD.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
    
print ('For testing data the RMSE is %s' % (error))

For testing data the RMSE is 0.8323818276407379


In [42]:
# 4 Get the complete movie data set
complete_movies_file = os.path.join(datasets_path, 'ml-latest', 'movies.csv')
complete_movies_raw_data = sc.textFile(complete_movies_file)
complete_movies_raw_data_header = complete_movies_raw_data.take(1)[0]

# Parse
complete_movies_data = complete_movies_raw_data.filter(lambda line: line!=complete_movies_raw_data_header)\
    .map(lambda line: line.split(",")).map(lambda tokens: (int(tokens[0]),tokens[1],tokens[2])).cache()

complete_movies_titles = complete_movies_data.map(lambda x: (int(x[0]),x[1]))
    
print ("There are %s movies in the complete dataset" % (complete_movies_titles.count()))

There are 45843 movies in the complete dataset


used groupbyKey

In [43]:
# 4 The complete rating data
def get_counts_and_averages(ID_and_ratings_tuple):
    nratings = len(ID_and_ratings_tuple[1])
    return ID_and_ratings_tuple[0], (nratings, float(sum(x for x in ID_and_ratings_tuple[1]))/nratings)

movie_ID_with_ratings_RDD = (complete_ratings_data.map(lambda x: (x[1], x[2])).groupByKey())
movie_ID_with_avg_ratings_RDD = movie_ID_with_ratings_RDD.map(get_counts_and_averages)
movie_rating_counts_RDD = movie_ID_with_avg_ratings_RDD.map(lambda x: (x[0], x[1][0]))

In [44]:
movie_ID_with_avg_ratings_RDD.take(3)

[(135168, (6, 3.1666666666666665)), (90112, (4, 2.25)), (65538, (10, 3.35))]

In [50]:
# 4 add new user with rating data
new_user_ID = 0

# The format of each line is (userID, movieID, rating)
new_user_ratings = [
     (0,260,4), # Star Wars (1977)
     (0,1,3), # Toy Story (1995)
     (0,16,3), # Casino (1995)
     (0,25,4), # Leaving Las Vegas (1995)
     (0,32,4), # Twelve Monkeys (a.k.a. 12 Monkeys) (1995)
     (0,335,1), # Flintstones, The (1994)
     (0,379,1), # Timecop (1994)
     (0,296,3), # Pulp Fiction (1994)
     (0,858,5) , # Godfather, The (1972)
     (0,50,4) # Usual Suspects, The (1995)
    ]
new_user_ratings_RDD = sc.parallelize(new_user_ratings)
print ('New user ratings: %s' % new_user_ratings_RDD.take(10))

New user ratings: [(0, 260, 4), (0, 1, 3), (0, 16, 3), (0, 25, 4), (0, 32, 4), (0, 335, 1), (0, 379, 1), (0, 296, 3), (0, 858, 5), (0, 50, 4)]


used RDD.union(RDD)

In [51]:
# 5 add the new usder data for training
complete_data_with_new_ratings_RDD = complete_ratings_data.union(new_user_ratings_RDD)


In [52]:
# 5 retrain the model
from time import time

t0 = time()
new_ratings_model = ALS.train(complete_data_with_new_ratings_RDD, best_rank, seed=seed, 
                              iterations=iterations, lambda_=regularization_parameter)
tt = time() - t0

print ("New model trained in %s seconds" % round(tt,3))

New model trained in 156.166 seconds


In [53]:
# 5. predict the user with other movie id's.
new_user_ratings_ids = map(lambda x: x[1], new_user_ratings) # get just movie IDs
# keep just those not on the ID list (thanks Lei Li for spotting the error!)
new_user_unrated_movies_RDD = (complete_movies_data.filter(lambda x: x[0] not in new_user_ratings_ids).map(lambda x: (new_user_ID, x[0])))

# Use the input RDD, new_user_unrated_movies_RDD, with new_ratings_model.predictAll() to predict new ratings for the movies
new_user_recommendations_RDD = new_ratings_model.predictAll(new_user_unrated_movies_RDD)

In [54]:
new_user_recommendations_RDD.take(3)

[Rating(user=0, product=116688, rating=0.7348019887496817),
 Rating(user=0, product=32196, rating=2.81062170660453),
 Rating(user=0, product=138744, rating=1.964685418787016)]

In [55]:
# 5 Transform new_user_recommendations_RDD into pairs of the form (Movie ID, Predicted Rating)
new_user_recommendations_rating_RDD = new_user_recommendations_RDD.map(lambda x: (x.product, x.rating))
new_user_recommendations_rating_title_and_count_RDD = \
    new_user_recommendations_rating_RDD.join(complete_movies_titles).join(movie_rating_counts_RDD)
new_user_recommendations_rating_title_and_count_RDD.take(3)

[(147456, ((-1.2359053415947776, 'The Legend of Longwood (2014)'), 2)),
 (27648, ((1.994881058473843, 'Bright Young Things (2003)'), 171)),
 (149508, ((2.8553080789124294, 'Spellbound (2011)'), 13))]

In [56]:
new_user_recommendations_rating_title_and_count_RDD = \
    new_user_recommendations_rating_title_and_count_RDD.map(lambda r: (r[1][0][1], r[1][0][0], r[1][1]))

In [57]:
top_movies = new_user_recommendations_rating_title_and_count_RDD.filter(lambda r: r[2]>=25).takeOrdered(25, key=lambda x: -x[1])

print ('TOP recommended movies (with more than 25 reviews):\n%s' %
        '\n'.join(map(str, top_movies)))

TOP recommended movies (with more than 25 reviews):
("Long Night's Journey Into Day (2000)", 4.046686099779802, 35)
('"Godfather', 3.9704161044057473, 57070)
('Frozen Planet (2011)', 3.9430972328632423, 322)
('"Godfather: Part II', 3.9166391395755134, 36679)
('"Enemies of Reason', 3.908744117688372, 29)
('Heimat - A Chronicle of Germany (Heimat - Eine deutsche Chronik) (1984)', 3.8634101999309864, 32)
('"Two Escobars', 3.8260847358929113, 63)
('Planet Earth (2006)', 3.8242371733551357, 754)
('Band of Brothers (2001)', 3.816944657598791, 284)
('Death on the Staircase (Soupçons) (2004)', 3.811827589033921, 95)
('"Not on Your Life (Verdugo', 3.8070321140775913, 26)
('"Lord of the Rings: The Fellowship of the Ring', 3.806566386570744, 56827)
('Pulp Fiction (1994)', 3.788951216637552, 87901)
('"Lord of the Rings: The Return of the King', 3.780718221174822, 51837)
('"Civil War', 3.7725339426314712, 400)
('Madagascar (2011)', 3.7720072964589506, 26)
('Star Wars: Episode V - The Empire Strikes

In [58]:
# 6 predict for a single movie
my_movie = sc.parallelize([(0, 500)]) # Quiz Show (1994)
individual_movie_rating_RDD = new_ratings_model.predictAll(new_user_unrated_movies_RDD)
individual_movie_rating_RDD.take(1)

[Rating(user=0, product=116688, rating=0.7348019887496817)]

In [59]:
# 7 save the model
from pyspark.mllib.recommendation import MatrixFactorizationModel

model_path = os.path.join('.', 'models', 'movie_lens_als')

# Save and load model
model.save(sc, model_path)
same_model = MatrixFactorizationModel.load(sc, model_path)

In [60]:
ls models/

movie_lens_als/
